In [1]:
import networkx as nx
import json
import pandas as pd

In [2]:
functions = pd.read_csv('DBs/Dataset-1/training_Dataset-1.csv')
functions.head(2)

,Unnamed: 0,idb_path,fva,func_name,start_ea,end_ea,bb_num,hashopcodes,project,library,arch,bit,compiler,version,optimizations
0,0,IDBs/Dataset-1/openssl/x86-clang-3.5-O0_libcry...,0x5e280,AES_bi_ige_encrypt,0x5e280,0x5e8b1,53,af5a269e11b9f60c4fc1e30434e1caeb73b8a83da6850d...,openssl,libcrypto.so.3,x,32,clang,3.5,O0
1,1,IDBs/Dataset-1/openssl/arm64-clang-9-O1_libcry...,0xe1084,AES_bi_ige_encrypt,0xe1084,0xe13a0,40,8182af65a0c6cf2a72425c95ea87d26a140bcaa4cf5b25...,openssl,libcrypto.so.3,arm,64,clang,9,O1


**idb_path**
Binary that function is part of

**fva**
First virtual address(?) - appears to be same as start_ea

**func_name**
Pulled from object dump file, just the function name

**start_ea/end_ea**
Addresses where function starts/ends

**bb_num**
Count of basic blocks in the function

**bb_list**
Semicolon-separated list of basic blocks

**hashopcodes**
Hash of... something

In [3]:
flowchart = pd.read_csv('DBs/Dataset-1/features/flowchart_Dataset-1.csv')
flowchart.head(2)

,idb_path,fva,func_name,start_ea,end_ea,bb_num,bb_list,hashopcodes
0,IDBs/Dataset-1/clamav/x64-clang-5.0-O2_clamsca...,0x402a60,mdprintf,0x402a60,0x402e59,51,0x402a60;0x402a7d;0x402abd;0x402b21;0x402b30;0...,a101cff7a6d5aeabc81b55d5d747fd5bbe3175abdf2dcf...
1,IDBs/Dataset-1/clamav/x64-clang-5.0-O2_clamsca...,0x402e60,logg_close,0x402e60,0x402e8e,5,0x402e60;0x402e6b;0x402e70;0x402e7c;0x402e8c,42a84857127dd18fa63b8cba3b4b531c8e5c6b66010072...


In [4]:
subset = flowchart.head(1)
subset['bb_num'] == len(subset['bb_list'][0].split(';'))
# Just verifying that bb_num is what we think it is

0    True
Name: bb_num, dtype: bool

In [5]:
subset['idb_path'][0]

'IDBs/Dataset-1/clamav/x64-clang-5.0-O2_clamscan.i64'

In [6]:
pairs = pd.read_csv('DBs/Dataset-1/pairs/validation/pos_validation_Dataset-1.csv')
# These are all the positive (matching) pairs --> "pos_" (negative is "neg_")
pairs.head(2)

,Unnamed: 0,idb_path_1,fva_1,func_name_1,idb_path_2,fva_2,func_name_2,db_type
0,0,IDBs/Dataset-1/zlib/arm64-clang-7-O0_libz.so.1...,0x235fc,detect_data_type,IDBs/Dataset-1/zlib/arm64-clang-9-Os_minigzip6...,0x219148,detect_data_type,XC
1,1,IDBs/Dataset-1/zlib/arm32-clang-7-O1_minigzip.i64,0x1ee74,detect_data_type,IDBs/Dataset-1/zlib/arm64-clang-9-Os_minigzip6...,0x219148,detect_data_type,XM


In [7]:
pairs_cmr = pd.read_csv('DBs/Dataset-1-CodeCMR/pairs/testing/pos_rank_testing_Dataset-1-CodeCMR.csv')
pairs.head(200)

,Unnamed: 0,idb_path_1,fva_1,func_name_1,idb_path_2,fva_2,func_name_2,db_type
0,0,IDBs/Dataset-1/zlib/arm64-clang-7-O0_libz.so.1...,0x235fc,detect_data_type,IDBs/Dataset-1/zlib/arm64-clang-9-Os_minigzip6...,0x219148,detect_data_type,XC
1,1,IDBs/Dataset-1/zlib/arm32-clang-7-O1_minigzip.i64,0x1ee74,detect_data_type,IDBs/Dataset-1/zlib/arm64-clang-9-Os_minigzip6...,0x219148,detect_data_type,XM
2,2,IDBs/Dataset-1/zlib/x64-clang-9-Os_minigzip64.i64,0x408b40,detect_data_type,IDBs/Dataset-1/zlib/arm64-clang-9-Os_minigzip6...,0x219148,detect_data_type,XM
3,3,IDBs/Dataset-1/zlib/mips64-clang-3.5-Os_minigz...,0x2d958,detect_data_type,IDBs/Dataset-1/zlib/arm64-clang-9-Os_minigzip6...,0x219148,detect_data_type,XA
4,4,IDBs/Dataset-1/zlib/x86-clang-3.5-O2_libz.so.1...,0xae60,detect_data_type,IDBs/Dataset-1/zlib/arm64-clang-9-Os_minigzip6...,0x219148,detect_data_type,XA
...,...,...,...,...,...,...,...,...
195,195,IDBs/Dataset-1/zlib/x64-clang-9-Os_minigzip.i64,0x4085e4,init_block,IDBs/Dataset-1/zlib/x86-clang-7-Os_minigzip.i64,0x804fabf,init_block,XC+XB
196,196,IDBs/Dataset-1/zlib/x64-clang-5.0-O0_libz.so.1...,0x115c0,init_block,IDBs/Dataset-1/zlib/x86-clang-7-Os_minigzip.i64,0x804fabf,init_block,XC+XB
197,197,IDBs/Dataset-1/zlib/mips32-clang-3.5-O1_minigz...,0x2b8dc,init_block,IDBs/Dataset-1/zlib/x86-clang-7-Os_minigzip.i64,0x804fabf,init_block,XM
198,198,IDBs/Dataset-1/zlib/arm64-clang-5.0-O1_libz.so...,0x17628,init_block,IDBs/Dataset-1/zlib/x86-clang-7-Os_minigzip.i64,0x804fabf,init_block,XM


In [8]:
f = open('DBs/Dataset-1/features/testing/acfg_disasm_Dataset-1_testing/arm32-gcc-9-Os_ncat_acfg_disasm.json')
o = json.load(f)
o2 = o[list(o.keys())[0]]

In [9]:
addresses = list(o2.keys())
addresses[:3]

['0x13e14', '0x31720', '0x1d744']

In [10]:
o2[addresses[18]]

{'elapsed_time': 0.0019669532775878906,
 'nodes': [81344, 81408, 81412, 81388, 81360, 81268, 81372],
 'edges': [[81388, 81408],
  [81360, 81372],
  [81268, 81372],
  [81344, 81360],
  [81388, 81412],
  [81268, 81344],
  [81360, 81388],
  [81344, 81372]],
 'basic_blocks': {'81344': {'bb_len': 16,
   'bb_mnems': ['ldr', 'ldrb', 'cmp', 'bne'],
   'bb_norm': ['ldr_r2,_[sp+0]', 'ldrb_r2,_[r2+0]', 'cmp_r2,_0x0', 'bne_himm'],
   'bb_disasm': ['ldr r2, [sp]',
    'ldrb r2, [r2]',
    'cmp r2, #0',
    'bne #0x13ddc'],
   'b64_bytes': 'ACCd5QAg0uUAAFLjAgAAGg==',
   'bb_heads': [81344, 81348, 81352, 81356]},
  '81408': {'bb_len': 4,
   'bb_mnems': ['bl'],
   'bb_norm': ['bl_himm'],
   'bb_disasm': ['bl #0x11f34'],
   'b64_bytes': 'S/j/6w==',
   'bb_heads': [81408]},
  '81412': {'bb_len': 8,
   'bb_mnems': ['add', 'pop'],
   'bb_norm': ['add_sp,_sp,_0xc', 'pop_r4,_r5,_r6,_r7,_pc'],
   'bb_disasm': ['add sp, sp, #0xc', 'pop {r4, r5, r6, r7, pc}'],
   'b64_bytes': 'DNCN4vCAveg=',
   'bb_heads': [81

**'bb_len'** Number of bytes of the basic block

**'bb_mnems'** Mnemonics: intructions

**'bb_norm'** Instruction with relative addresses

**'bb_disasm'** Disassembly: The actual disassembled code

**'b64_bytes'** A string that represents the bytes (encoded) (guessing)

**'bb_heads'** Addresses where the basic block starts (we think.. why are these always 4 apart?)

In [19]:
#for n in addresses:
n = addresses[0]
network = o2[n]
G = nx.Graph()
G.add_nodes_from(network["nodes"])
G.add_edges_from(network["edges"])
G
nx.write_graphml(G,'graphs/graph.xml')

In [20]:
D = nx.DiGraph()
D.add_nodes_from(network["nodes"])
D.add_edges_from(network["edges"])
nx.write_graphml(D,'graphs/digraph.xml')

In [13]:
from pyvis.network import Network

In [14]:
def make_network_graph():
    net = Network()
    nodes = network["nodes"]
    net.add_nodes(nodes)
    for edge in network["edges"]:
        net.add_edge(edge[0], edge[1])
    net.toggle_physics(True)
    net.show(f'network.html')

In [15]:
net = Network()
nodes = network["nodes"]
net.add_nodes(nodes)
for edge in network["edges"]:
    net.add_edge(edge[0], edge[1])
net.toggle_physics(True)
net.show(f'graphs/network.html')

In [16]:
!pwd

/Users/janeadams/projects/class/ml/nba


In [17]:
net.show('test.html')

In [18]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))


KeyError: 'pos'